In [99]:
#import voor alle benodigde libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import seaborn as sns
import plotly.express as px
from sodapy import Socrata

In [100]:
#Inladen API voertuigen
#Van de site:
#https://opendata.rdw.nl/Voertuigen/Open-Data-RDW-Gekentekende_voertuigen/m9d7-ebf2
client_1 = Socrata("opendata.rdw.nl", None)

voertuigen = client_1.get("m9d7-ebf2", limit=50000, where='laadvermogen>0')

voertuigen_df = pd.DataFrame.from_records(voertuigen)

voertuigen_df.shape

(50000, 79)

In [101]:
#Inladen API brandstof types
#Van de site:
#https://opendata.rdw.nl/Voertuigen/Open-Data-RDW-Gekentekende_voertuigen_brandstof/8ys7-d773
client_2 = Socrata("opendata.rdw.nl", None)

brandstof = client_1.get("8ys7-d773", limit=1300000, where="brandstof_omschrijving='Elektriciteit'")

brandstof_df = pd.DataFrame.from_records(brandstof)

brandstof_df.head()

,kenteken,brandstof_volgnummer,brandstof_omschrijving,emissiecode_omschrijving,klasse_hybride_elektrisch_voertuig,uitlaatemissieniveau,geluidsniveau_rijdend,nominaal_continu_maximumvermogen,netto_max_vermogen_elektrisch,max_vermogen_60_minuten,...,brandstofverbruik_stad,co2_uitstoot_gewogen,geluidsniveau_stationair,actie_radius_extern_opladen_wltp,elektrisch_verbruik_extern_opladen_wltp,actie_radius_extern_opladen_stad_wltp,brandstof_verbruik_gewogen_gecombineerd_wltp,emis_co2_gewogen_gecombineerd_wltp,brandstof_verbruik_gecombineerd_wltp,emis_deeltjes_type1_wltp
0,00BBN1,2,Elektriciteit,Z,OVC-HEV,EURO 5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00BLT6,1,Elektriciteit,Z,NaN,NaN,78,210.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00BLZ9,1,Elektriciteit,Z,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00BNX6,1,Elektriciteit,Z,NaN,NaN,NaN,123.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00BPV3,1,Elektriciteit,Z,NaN,NaN,72,160.00,205.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
#Inladen API chargemap
response = requests.get("https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&maxresults=1000&compact=true&verbose=false&key=6686970f-0aa6-4cb5-ae81-cf8f6389a816")

#Omzetten naar dictionary
responsejson  = response.json()

#Dataframe bevat kolom die een list zijn. 
#Met json_normalize zet je de eerste kolom om naar losse kolommen
charge_map = pd.json_normalize(responsejson)

charge_map.head()

,IsRecentlyVerified,DateLastVerified,ID,UUID,DataProviderID,OperatorID,UsageTypeID,Connections,NumberOfPoints,StatusTypeID,...,GeneralComments,AddressInfo.RelatedURL,UsageCost,AddressInfo.AddressLine2,AddressInfo.ContactTelephone1,AddressInfo.AccessComments,AddressInfo.ContactEmail,AddressInfo.ContactTelephone2,OperatorsReference,DataProvidersReference
0,True,2023-03-15T11:19:00Z,256957,C274DD57-63EE-4953-8723-61A928DC8A64,1,1.0,6.0,[],1.0,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,2023-03-04T15:56:00Z,238390,54DE6F07-8F83-4D6C-AC5C-F46C3CC5AF40,1,47.0,4.0,"[{'ID': 392175, 'ConnectionTypeID': 33, 'Statu...",1.0,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,2023-03-04T15:52:00Z,238389,2A59654E-C064-4234-AD9F-9E814387FE0B,1,47.0,4.0,"[{'ID': 392173, 'ConnectionTypeID': 33, 'Statu...",6.0,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,2023-03-04T15:49:00Z,238388,2C11156F-072A-416A-9489-B2D96889BD9F,1,47.0,4.0,"[{'ID': 392171, 'ConnectionTypeID': 33, 'Statu...",1.0,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,2023-03-04T14:27:00Z,238387,7CC524E9-1377-47C2-97A5-B616B76DD825,1,47.0,4.0,"[{'ID': 392170, 'ConnectionTypeID': 33, 'Statu...",2.0,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
#laadpaal = pd.read_csv('laadpaaldata.csv')

NOTE TO SELF: NA DE MERGE DOWNLOADEN ALS CSV, DAN HOEFT JE NIET STEEDS 16 MILJOEN TE RUNNEN